In [60]:
# Import Libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [61]:
# Constants
# Data Location
LOC = "D:\DSCI\Limited_set"
Model_LOC = ""
# Classes to use
classes = {

}

In [62]:
# Model Dictionary
models = {
    "Xception": tf.keras.applications.xception.Xception(weights="imagenet"),
    "VGG16": tf.keras.applications.vgg16.VGG16(weights="imagenet"),
    "VGG19": tf.keras.applications.vgg19.VGG19(weights="imagenet"),
    "InceptionV3": tf.keras.applications.InceptionV3(weights="imagenet"),
    "ResNet50": tf.keras.applications.resnet50.ResNet50(weights="imagenet"),
#    "AlexNet": tf.keras.models.load_model("")
}



In [65]:
# Testing
def test_model(model):
    print("Running tests on model:", model)
    a = models[model].predict(train)
    b = models[model].predict(test)
    print(a)


In [65]:
for model1 in models:
    shapes = models[model1].get_config()["layers"][0]["config"]["batch_input_shape"]
    shape = (shapes[1], shapes[2])
    train, test = tf.keras.preprocessing.image_dataset_from_directory(LOC, validation_split=.2, subset="both", seed=42, image_size=shape)

    try:
        test_model(model1)
    except Exception as e:
        print(e)


Found 6500 files belonging to 5 classes.
Using 5200 files for training.
Using 1300 files for validation.
Running tests on model: Xception
 12/163 [=>............................] - ETA: 5:13